In [190]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

df = pd.read_csv('loan_data.csv')

np.random.seed(42)

df = df.drop(columns=['Loan_ID'], axis=1)
df = df.dropna(axis=0)

In [191]:
def Categorical_Columns(data):
    categoricalValues = []
    for i in range(len(data.columns)):
        try:
            int(data[data.columns[i]][0])
            continue
        except:
            categoricalValues.append(data.columns[i])
    return categoricalValues

categorical_columns = Categorical_Columns(df)
print(categorical_columns)

categorical_to_int = [[1, 0], [1, 0], [1, 0, 2, 3, 5], [1, 0], [0, 1], [1, 3, 2], [0, 1]]

def NormalizationData(data, categorical_columns, categorical_to_int):
    normalized_data = data.copy()
    for i in range(len(normalized_data.columns)):
        if(data.columns[i] in categorical_columns or data.columns[i] == 'Dependents'):
            normalized_data = normalized_data.replace((normalized_data[normalized_data.columns[i]].unique()), (categorical_to_int[0]))
            categorical_to_int = categorical_to_int[1:]
        normalized_data[normalized_data.columns[i]] = normalized_data[normalized_data.columns[i]] / max(normalized_data[normalized_data.columns[i]])
    return normalized_data

normalized_data = NormalizationData(df, categorical_columns, categorical_to_int)

['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area', 'Loan_Status']


In [192]:
xcol = list(df.drop(columns = ['Loan_Status']).columns)
ycol = ['Loan_Status']

x_train, test0= train_test_split(normalized_data, test_size=0.2)
test, val = train_test_split(test0, test_size=0.5)

accuracy = []
def TrainingModel(classifierType):

    classifierType.fit(x_train[xcol], x_train[ycol])
    prediction = classifierType.predict(test[xcol])

    score = accuracy_score(test[ycol], prediction)
    accuracy.append(score)
    return (accuracy, score, prediction)

(svm, rand_forest, decision_tree, log_regression) = (svm.SVC(), RandomForestClassifier(), DecisionTreeClassifier(), LogisticRegression())
classifiers = [svm, rand_forest, decision_tree, log_regression]

best_prediction_data = []
best_accuracy = -1

for i in range(len(classifiers)):
    (data_model, acc, prediction) = TrainingModel(classifiers[i])
    print(f"\033[1;30;46mClassifier: {classifiers[i]}\tAccuracy: {acc}\033[0m")
    if(best_accuracy < acc):
        best_accuracy = acc
        best_prediction_data = [] 
        best_prediction_data.append((classifiers[i], acc))

print(f"\033[1;30;42mBest model: {best_prediction_data[0][0]}\tScore: {best_prediction_data[0][1]}\033[0m")

Classifier: SVC()	Accuracy: 0.8064516129032258


c:\Users\Ern\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Ern\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Classifier: RandomForestClassifier()	Accuracy: 0.8064516129032258
Classifier: DecisionTreeClassifier()	Accuracy: 0.8709677419354839
Classifier: LogisticRegression()	Accuracy: 0.8064516129032258
Best model: DecisionTreeClassifier()	Score: 0.8709677419354839


c:\Users\Ern\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [193]:
#Saving to csv

final_data = pd.DataFrame(data = {'Predictions': prediction, 'Real values': list(test[ycol].reset_index().drop(columns=['index'])['Loan_Status'])})
final_data['Predictions'] = final_data['Predictions'].astype(int)
final_data['Real values'] = final_data['Real values'].astype(int)

new_file = pd.DataFrame(data = {'Predictions': list(final_data['Predictions'] ), 'Real value': list(final_data['Real values'])})

new_file.to_csv('Loan_prediction.csv', index=False)